Допустим, у нас имеется следующий класс для создания точки.

In [1]:
class Point3D:
    def __init__(self, x, y, z):
        self._x = x
        self._y = y
        self._z = z
        
    @classmethod
    def verify_coord(cls, coord):
        if type(coord) != int:
            raise TypeError("Координаты должны быть целым числом")
        
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, value):
        self.verify_coord(value)
        self._x = value
        
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, value):
        self.verify_coord(value)
        self._y = value
    
    @property
    def z(self):
        return self._z
    
    @z.setter
    def z(self, value):
        self.verify_coord(value)
        self._z = value
        
p = Point3D(1, 2, 3)
print(p.__dict__)

{'_x': 1, '_y': 2, '_z': 3}


Во-первых, мы можем сразу воспользоваться нашими декораторами, а именно, сеттерами и геттерами прямо в инициализаторе и переписать код следующим образом:

In [4]:
class Point3D:
    def __init__(self, x, y, z):
        # мы можем в инициализаторе сразу же воспользовать объектами свойств, то есть каждый раз будет срабатывать
        # соответствующий декоратор сеттер
        self.x = x
        self.y = y
        self.z = z
        
    @classmethod
    def verify_coord(cls, coord):
        if type(coord) != int:
            raise TypeError("Координаты должны быть целым числом")
        
    @property
    def x(self):
        return self._x
    
    @x.setter
    def x(self, value):
        self.verify_coord(value)
        self._x = value
        
    @property
    def y(self):
        return self._y
    
    @y.setter
    def y(self, value):
        self.verify_coord(value)
        self._y = value
    
    @property
    def z(self):
        return self._z
    
    @z.setter
    def z(self, value):
        self.verify_coord(value)
        self._z = value
        
p = Point3D(1, 2, 3)
print(p.__dict__)

{'_x': 1, '_y': 2, '_z': 3}


В примере выше мы можем сразу же отметить функциональное дублирование кода, где у нас сохраняется общая логика для создания свойст, где меняется только имя атрибута(свойства). Оптимизировать данный код и поведение могут помочь 
**дескрипторы**.

**Дескриптор** - представляет собой класс, который содержит в себе специальный магический метод __ get __ , если мы говорит о non-data дескрипторах, а также магические методы __ set __ и __ del __ , если мы имеем ввиду data дескрипторы.

In [5]:
# non-data 

class A: 
    def __get__(self, instance, owner):
        return ...

# data    
class B:
    def __get__(self, instance, owner):
        return ...
    
    def __set__(self, instance, value):
        return ...
    
    def __del__(self):
        return ...

In [ ]:
class Integer:
    def __set_name__(self, name):
        self.name = '_' + name
        
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        print(f'__set__: {self.name} = {value}')
        instance.__dict__[self.name] = value


In [ ]:
class Point3D:
    
    x = Integer()
    y = Integer()
    z = Integer()
    
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

Эти атрибуты и есть дескрипторы данных, через которые будет проходить взаимодействие. Итак, когда мы создавали экземпляры классов Integer, то автоматически вызывался магический метод  __ set_name __ , в котором параметр self являлся ссылкой на создаваемый экземпляр класса; owner – ссылка на класс Point3D; name – имя атрибута (для первого объекта x, затем, y и z). В этом методе мы формируем локальное свойство с именем атрибута, добавляя перед ним одно нижнее подчеркивание (так принято делать при определении дескрипторов). В итоге, в экземплярах классов будут храниться имена _ x, _ y, _ z.

In [6]:
pt = Point3D(1, 2, 3)

Сработает инициализатор, а в нем идет обращение к дескрипторам x, y, z. В частности, мы им присваиваем переданные значения. В этом случае, в классе Integer срабатывает сеттер (магический метод __ set __ ), параметр self – это ссылка на объект дескриптора; instance – ссылка на объект pt, из которого произошло обращение к дескриптору; value – присваиваемое значение. В этом сеттере мы выводим в консоль сообщение, что был вызван данный метод и отображаем сохраненное имя и присваиваемое значение. Следующей строчкой через ссылку instance, то есть, на экземпляр класса pt, формируем в нем локальное свойство с именем self.name и присваиваем значение value. В результате, в объекте pt появляются локальные свойства _ x, _ y, _ z с соответствующими значениями.

In [7]:
print(pt.__dict__)

{'_x': 1, '_y': 2, '_z': 3}


In [8]:
pt.x

1

In [9]:
class Integer:
    
    @classmethod
    def verify_coord(cls, coord):
        if type(coord) != int:
            raise TypeError("Значение должно быть целым числом")
    
    def __set_name__(self, name):
        self.name = '_' + name
        
    def __get__(self, instance, owner):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        print(f'__set__: {self.name} = {value}')
        instance.__dict__[self.name] = value

In [ ]:
class Point3D:
    
    x = Integer()
    y = Integer()
    z = Integer()
    
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z